In [113]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as mlt
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns

In [114]:
def plot_imputation_results(real_dataset, generated_dataset1, generated_dataset2, mask, startindex, endindex, path):

    tr = real_dataset[startindex:endindex, 1]
    te = generated_dataset1[startindex:endindex, 1]
    tx = generated_dataset2[startindex:endindex, 1]
    mask1 = mask[startindex:endindex, 1]
    te = (mask1 * te) + (1 - mask1) * tr
    tx = (mask1 * tx) + (1 - mask1) * tr
    mlt.figure(figsize=(20, 6))
    y = np.arange(1, len(tr) + 1)

    start_idx = 0
    for i in range(len(te)-1):
        color = "green"
        mlt.plot(y[start_idx:i+2],te[start_idx:i+2], color=color, linewidth = '3')
        mlt.plot(y[start_idx:i+2],tx[start_idx:i+2], color='red', linewidth = '2')
        start_idx = i + 1

    start_idx = 0
    for i in range(len(te)-1):
        color = "green"
        mlt.plot(y[start_idx:i+2],tx[start_idx:i+2], color='red', linewidth = '2')
        start_idx = i + 1

    start_idx = 0
    for i in range(len(tr)-1):
        color = "blue"
        if mask1[i] == 0:
            mlt.plot(y[start_idx:i+2],tr[start_idx:i+2], color=color)
        elif mask1[i] == 1:
            color = 'yellow'
            mlt.plot(y[start_idx:i+2],tr[start_idx:i+2], color=color, linestyle='--')
        start_idx = i + 1


    mlt.suptitle('Gan prediction on training dataset')
    mlt.ylabel('Energy Consumption in Kwh')
    mlt.legend()
    mlt.savefig(path)
    mlt.show()

In [115]:
def plot_imputation_results_two(real_dataset, generated_dataset1, mask, startindex, endindex, path):

    tr = real_dataset[startindex:endindex, 1]
    te = generated_dataset1[startindex:endindex, 1]
    mask1 = mask[startindex:endindex, 1]
    te = (mask1 * te) + (1 - mask1) * tr
    mlt.figure(figsize=(20, 6))
    y = np.arange(1, len(tr) + 1)

    start_idx = 0
    for i in range(len(te)-1):
        color = "green"
        mlt.plot(y[start_idx:i+2],te[start_idx:i+2], color=color, linewidth = '3')
        start_idx = i + 1

    start_idx = 0
    for i in range(len(tr)-1):
        color = "blue"
        if mask1[i] == 0:
            mlt.plot(y[start_idx:i+2],tr[start_idx:i+2], color=color)
        elif mask1[i] == 1:
            color = 'red'
            mlt.plot(y[start_idx:i+2],tr[start_idx:i+2], color=color, linestyle='--')
        start_idx = i + 1


    mlt.suptitle('Gan prediction on training dataset')
    mlt.ylabel('Energy Consumption in Kwh')
    mlt.legend()
    mlt.savefig(path)
    mlt.show()

In [116]:
def plot_full_dataset(real_dataset, gen_dataset, startindex, endindex, path, col):
    tr = real_dataset[startindex:endindex, col]
    te = gen_dataset[startindex:endindex, col]
    mlt.figure(figsize=(20, 6))
    mlt.suptitle('Gan prediction on test dataset')
    mlt.ylabel('Energy Consumption in Kwh')
    mlt.plot(tr, label='real')
    mlt.plot(te, label='gen')
    mlt.legend()
    mlt.savefig(path)
    mlt.show()

In [117]:
def compare_predicted_dataset(real_dataset, gen_dataset_mogrifier, gen_dataset_bilstm, gen_dataset_lstm, startindex, endindex, path, col):
    tr = real_dataset[startindex:endindex, col]
    te1 = gen_dataset_mogrifier[startindex:endindex, col]
    te2 = gen_dataset_bilstm[startindex:endindex, col]
    te3 = gen_dataset_lstm[startindex:endindex, col]
    mlt.figure(figsize=(20, 10))
    mlt.suptitle('Imputation comparison on office dataset')
    mlt.ylabel('Energy Consumption in Kwh')
    mlt.plot(tr, label='True value', linestyle='--', linewidth = '3')
    mlt.plot(te1, label='Seq2Seq', linewidth = '2')
    mlt.plot(te2, label='Knn', linewidth = '2')
    mlt.plot(te3, label='Mean')
    mlt.legend()
    mlt.savefig(path)
    mlt.show()

In [118]:
def plot_r2_score(results, path):
    # Prepare the data for plotting
    forecasting_models = list(results[next(iter(results))].keys())
    imputation_methods = list(results.keys())
    mae_values = {impute: [results[impute][forecast] for forecast in forecasting_models] for impute in imputation_methods}

    # Generate the plot
    fig, ax = mlt.subplots(figsize=(10, 6))

    for impute, values in mae_values.items():
        line = ax.plot(forecasting_models, values, marker='o', label=impute)
        for i, value in enumerate(values):
            ax.annotate(f'{value:.2f}',
                        xy=(forecasting_models[i], value),
                        xytext=(5, 5),  # 5 points offset
                        textcoords='offset points',
                        ha='center',
                        va='bottom')

    ax.set_xlabel('Forecasting Model')
    ax.set_ylabel('Avg R2 Score')
    ax.set_title('R2 results of Palo Alto dataset in different horizons')
    ax.legend()

    mlt.tight_layout()
    mlt.savefig(path)
    mlt.show()

In [119]:
def plot_forecasting_evaluation_results():
    # Data
    forecasting_models = ['Bilstm', 'Mogrifier LSTM', 'Scinet']
    imputation_methods = ['Real', 'ResiDualNet', 'knn']

    rmse_values = {
        'Real': [14.29, 13.909, 10.41],
        'ResiDualNet': [14.94, 14.215, 11.23],
        'knn': [16.04, 15.312, 13.62]
    }

    mse_values = {
        'Real': [204.42, 193.466, 108.45],
        'ResiDualNet': [223.20, 202.06, 126.11],
        'knn': [257.38, 234.47, 185.71]
    }

    r2_values = {
        'Real': [0.83, 0.84, 0.91],
        'ResiDualNet': [0.81, 0.81, 0.89],
        'knn': [0.75, 0.77, 0.82]
    }

    mae_values = {
        'Real': [9.19, 9.199, 6.91],
        'ResiDualNet': [9.21, 9.680, 7.02],
        'knn': [10.71, 10.331, 9.70]
    }

    # Define marker styles for each dataset
    marker_styles = {'Real': 's', 'ResiDualNet': '^', 'knn': 'o'}

    # Plot RMSE
    mlt.figure(figsize=(10, 6))
    mlt.title('RMSE results')
    for impute, values in rmse_values.items():
        mlt.plot(forecasting_models, values, marker=marker_styles[impute], label=impute)
        for i, value in enumerate(values):
            mlt.annotate(f'{value:.2f}',
                        xy=(forecasting_models[i], value),
                        xytext=(5, 5),  # 5 points offset
                        textcoords='offset points',
                        ha='center',
                        va='bottom',
                        fontsize=12)
    mlt.xlabel('Forecasting Model')
    mlt.ylabel('RMSE')
    mlt.legend()
    mlt.grid(True)
    mlt.tight_layout()
    #plt.savefig('rmse_plot.png')
    mlt.show()

    # Plot MSE
    mlt.figure(figsize=(10, 6))
    mlt.title('MSE results')
    for impute, values in mse_values.items():
        mlt.plot(forecasting_models, values, marker=marker_styles[impute], label=impute)
        for i, value in enumerate(values):
            mlt.annotate(f'{value:.2f}',
                        xy=(forecasting_models[i], value),
                        xytext=(5, 5),  # 5 points offset
                        textcoords='offset points',
                        ha='center',
                        va='bottom',
                        fontsize=12)
    mlt.xlabel('Forecasting Model')
    mlt.ylabel('MSE')
    mlt.legend()
    mlt.grid(True)
    mlt.tight_layout()
    #plt.savefig('mse_plot.png')
    mlt.show()

    mlt.figure(figsize=(10, 6))
    mlt.title('MAE results')
    for impute, values in mae_values.items():
        mlt.plot(forecasting_models, values, marker=marker_styles[impute], label=impute)
        for i, value in enumerate(values):
            mlt.annotate(f'{value:.2f}',
                        xy=(forecasting_models[i], value),
                        xytext=(5, 5),  # 5 points offset
                        textcoords='offset points',
                        ha='center',
                        va='bottom',
                        fontsize=12)
    mlt.xlabel('Forecasting Model')
    mlt.ylabel('MAE')
    mlt.legend()
    mlt.grid(True)
    mlt.tight_layout()
    #plt.savefig('mae_plot.png')
    mlt.show()

    # Plot R2
    mlt.figure(figsize=(10, 6))
    mlt.title('R² results')
    for impute, values in r2_values.items():
        mlt.plot(forecasting_models, values, marker=marker_styles[impute], label=impute)
        for i, value in enumerate(values):
            mlt.annotate(f'{value:.2f}',
                        xy=(forecasting_models[i], value),
                        xytext=(5, 5),  # 5 points offset
                        textcoords='offset points',
                        ha='center',
                        va='bottom',
                        fontsize=12)
    mlt.xlabel('Forecasting Model')
    mlt.ylabel('R²')
    mlt.legend()
    mlt.grid(True)
    mlt.tight_layout()
    #plt.savefig('r2_plot.png')
    mlt.show()


In [120]:
def get_ev_demand_graph(y_data, x_data):
    mlt.figure(figsize=(4, 3))  # Set the figure size (height, width)

    # Create horizontal bar plot
    bars = mlt.barh(x_data, y_data, color='skyblue')

    for bar, value in zip(bars, y_data):
        mlt.text(bar.get_width() / 2.3, bar.get_y() + bar.get_height()/2, f'{value:.1f}', va='center', color='black')

    # Adding labels and title
    mlt.ylabel('Year')
    mlt.xlabel('EVs sold in Millions')
    mlt.title('EV sales in US')

    mlt.grid(axis='x', linestyle='--', alpha=0.7)
    # Display the plot
    #plt.tight_layout()  # Adjust layout to prevent clipping of labels
    mlt.show()


In [110]:
# values_in_millions = [0.2, 0.2, 0.4, 0.3, 0.3, 0.6, 1, 1.6]
# years = ['2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
# get_ev_demand_graph(values_in_millions, years)